In [1]:
:!./install.sh

# 컴파일러에서 변수, 조건문 다루기
TODO

In [2]:
data Expr = Var Name
          | Val Value
          | Add Expr Expr
          -- | Sub Expr Expr
          -- | Mul Expr Expr
          -- | Div Expr Expr
          | If Expr Expr Expr
          deriving Show

type Name = String
type Value = Int

data Inst = ADD | PUSH Value | GOTO Code | JMPZ Code | JUMP Code | READ Addr  deriving Show
type Code = [Inst]

type Addr = Int

type Env = [ (Name, Value) ]
type SymTbl = [ (Name, Addr) ]
type Memory = [ (Addr, Value) ]

type Stack = [Value]
type Kont = (Stack,Memory,Code) -> (Stack,Memory,Code)

haltK :: Kont
haltK (s, mem, _) = (s, mem, [])

pushK :: Int -> Kont
pushK n (s, mem, code) = (n:s, mem, code)

addK :: Kont
addK (n2:n1:s, mem, code) = ((n1+n2):s, mem, code)

jmpzK :: Code -> Kont
jmpzK code (0:s, mem, _) = (s, mem, code)
jmpzK _    (_:s, mem, c) = (s, mem, c)

jumpK :: Code -> Kont
jumpK code (s, mem, _) = (s, mem, code)

readK a (s, mem, code) = case lookup a mem of
                           Nothing -> error (show a ++ " uninitialized memory address")
                           Just v  -> (v:s, mem, code)

compile :: SymTbl -> Expr -> Code
compile tbl (Var x) = case lookup x tbl of
                        Nothing -> error (x ++ " not found")
                        Just a  -> [READ a]
compile tbl (Val n) = [PUSH n]
compile tbl (Add e1 e2) = compile tbl e1 ++ compile tbl e2 ++ [ADD]
compile tbl (If e e1 e0) = compile tbl e ++ [JMPZ c0] ++ c1 ++ [GOTO []] ++ c0
                         where
                           c1 = compile tbl e1
                           c0 = compile tbl e0

step :: Inst -> Kont
step (PUSH n) = pushK n
step ADD      = addK
step (GOTO c) = jumpK c
step (JMPZ c) = jmpzK c
step (READ a) = readK a

run :: Kont
run m@(s, mem, []) = m
run (s, mem, c:cs) = run $ step c (s, mem, cs)

In [3]:
{-  b = ?,  x = 2, y = 4

    if b then (x + 3) else y  -}

e1 = If (Var "b") (Add (Var "x") (Val 3)) (Var "y")
e2 = e1 `Add` Val 200

import Data.List (union)

vars (Var x) = [x]
vars (Val _) = []
vars (Add e1 e2) = vars e1 `union` vars e2
vars (If e e1 e0) = vars e `union` vars e1 `union` vars e0

vars e1
vars e2


tbl0 = [("b",101),("x",102),("y",103)]
tbl0

mem0 = [(101,0), (102,2), (103,3)]
mem0

code1 = compile tbl0 e1
code1

code2 = compile tbl0 e2
code2

vm0@(s0,   _,c0:cs0) = ([], mem0, code2)
vm0
vm1@(s1,mem1,c1:cs1) = step c0 (s0,mem0,cs0)
vm1
vm2@(s2,mem2,c2:cs2) = step c1 (s1,mem1,cs1)
vm2
-- vm3@(s3,mem3,c3:cs3) = step c2 (s2,mem2,cs2)
-- vm3

run vm0

import GHC.HeapView

putStr =<< ppHeapGraph <$> buildHeapGraph 15 code2 (asBox code2)

["b","x","y"]

["b","x","y"]

[("b",101),("x",102),("y",103)]

[(101,0),(102,2),(103,3)]

[READ 101,JMPZ [READ 103],READ 102,PUSH 3,ADD,GOTO [],READ 103]

...

([],[(101,0),(102,2),(103,3)],[READ 101,JMPZ [READ 103],READ 102,PUSH 3,ADD,GOTO [],READ 103,PUSH 200,ADD])

([0],[(101,0),(102,2),(103,3)],[JMPZ [READ 103],READ 102,PUSH 3,ADD,GOTO [],READ 103,PUSH 200,ADD])

([],[(101,0),(102,2),(103,3)],[READ 103])

([3],[(101,0),(102,2),(103,3)],[])

let x1 = READ (I# 103)
    x2 = [] 1104125280
    x3 = _bh (ADD 283491905952)
in _bh (READ (I# 101) : JMPZ (x1 : x2) : READ (I# 102) : PUSH (_bh (I# 3)) : x3 : GOTO x2 : x1 : PUSH (_bh (I# 200)) : x3 : x2)